# Getting Started
Before we can build a model we need data. We'll use the [Stack Exchange API](https://api.stackexchange.com/) and the Python package [`StackAPI`](https://stackapi.readthedocs.io/en/latest/).

> NOTE: Before installing I suggest creating a new environment so as not to mess up any of your other dependencies. If you don't know how to do this you can see my blog, [_We Recommend Creating an Environment_](https://medium.com/@ianiat11/we-recommend-creating-an-environment-da38af0cecbb).

# The API
The `StackAPI` package can be used to query the Stack Exchange API -- meaning any of the subdomains of [Stack Exchange](https://stackexchange.com/). I'm only interested in [Stack Overflow](https://stackoverflow.com/), so I'm going to write a small package that limits the usage of the `StackAPI` to that site.

We'll start by creating a new Python package called `is_question`. This will eventually hold all of our source code for our project. To make the empty package make a directory called "is_question" and put an "\_\_init\_\_.py" file inside of it. Boom! Now you have an `is_question` Python package. &#128526; Inside the package add another file called "api.py". This is where we'll define our own `StackOverflow` class that only makes API calls to the Stack Overflow site. Now add the following to "api.py":

In [4]:
"""contents of api.py"""
from stackapi import StackAPI


class StackOverflow(StackAPI):
    """A subclass of `StackAPI` that limits API calls to StackOverflow."""

We have now subclassed the `StackAPI` class. Next we limit the site.

>NOTE: If you're unfamiliar with **O**bject-**O**riented **P**rogramming (**OOP**) I suggest watching [this tutorial series](https://www.youtube.com/watch?v=ZDa-Z5JzLYM&list=PL-osiE80TeTsqhIuOqKhwlXsIBIdSeYtc&pp=iAQB) from [Corey Shafer](https://www.youtube.com/@coreyms/featured), or any other OOP tutorial you prefer.

In [5]:
"""contents of api.py"""
from stackapi import StackAPI


class StackOverflow(StackAPI):
    """A subclass of `StackAPI` that limits API calls to StackOverflow."""

    def __init__(self, **kwargs):
        super().__init__(name="stackoverflow", **kwargs)

Within the `__init__` method we hard-coded the `name` argument as `"stackoverflow"`. If we were to use the `StackAPI` class as-is we'd do this:

In [7]:
so = StackAPI(name="stackoverflow")

# fetch some questions
response = so.fetch("questions")
print(response)

{'backoff': 0, 'has_more': True, 'page': 5, 'quota_max': 300, 'quota_remaining': 291, 'total': 0, 'items': [{'tags': ['python', 'web-scraping'], 'owner': {'reputation': 29, 'user_id': 20778783, 'user_type': 'registered', 'profile_image': 'https://www.gravatar.com/avatar/b51d91aa360bf6aeb07061d7cd6820e0?s=256&d=identicon&r=PG', 'display_name': 'mario', 'link': 'https://stackoverflow.com/users/20778783/mario'}, 'is_answered': True, 'view_count': 73, 'answer_count': 2, 'score': 1, 'last_activity_date': 1688236251, 'creation_date': 1688112636, 'last_edit_date': 1688124963, 'question_id': 76587144, 'content_license': 'CC BY-SA 4.0', 'link': 'https://stackoverflow.com/questions/76587144/is-there-any-way-to-scrape-this-website-faster-with-python', 'title': 'is there any way to scrape this website faster with python?'}, {'tags': ['set', 'brute-force', 'heuristics', 'np', 'picking'], 'owner': {'reputation': 1, 'user_id': 22159619, 'user_type': 'registered', 'profile_image': 'https://www.gravata

But because we've hard-coded the name, we don't have to specify it during instantiation:

In [10]:
so = StackOverflow()

# fetch some questions
response = so.fetch("questions")
print(response)

{'backoff': 0, 'has_more': True, 'page': 5, 'quota_max': 300, 'quota_remaining': 279, 'total': 0, 'items': [{'tags': ['python', 'python-3.x'], 'owner': {'reputation': 1, 'user_id': 22157891, 'user_type': 'registered', 'profile_image': 'https://lh3.googleusercontent.com/a/AAcHTtc8tgLlY310rmB7KNI7KIPGb2WRGCJxJhW5iPtDoSc=k-s256', 'display_name': 'averageConsumer', 'link': 'https://stackoverflow.com/users/22157891/averageconsumer'}, 'is_answered': False, 'view_count': 16, 'answer_count': 0, 'score': 0, 'last_activity_date': 1688236578, 'creation_date': 1688184618, 'last_edit_date': 1688236578, 'question_id': 76593242, 'content_license': 'CC BY-SA 4.0', 'link': 'https://stackoverflow.com/questions/76593242/controlling-mouse-with-python-not-working', 'title': 'Controlling mouse with Python Not working'}, {'tags': ['node.js', 'npm', 'create-react-app', 'deprecated'], 'owner': {'reputation': 1, 'user_id': 22159103, 'user_type': 'registered', 'profile_image': 'https://www.gravatar.com/avatar/49